In [6]:
import math
from openpyxl import load_workbook
from unipath import Path
import os
from django.core import exceptions

base = Path('d:\\', 'Users', 'javier', 'tmp')
r06 = Path(base, 'mac_06.xlsx')
r07 = Path(base, 'mac_07.xlsx')
r07b = Path(base, 'mac_07b.xlsx')
# wb6 = load_workbook(filename = r06)
# wb7 = load_workbook(filename = r07)
# wb7b = load_workbook(filename = r07b)
# ws6 = wb6['2021_06']
# ws7 = wb7['2021_06']
# ws7b = wb7b['2021_06']

In [7]:
def get_int(celda):
    """Convierte el valor de una celda en entero"""
    if celda.value == '':
        return 0
    try:
        if celda.ctype != 2:
            return 0
        return math.ceil(celda.value)
    except ValueError:
        return 0


def get_float(celda):
    """"Convierte el valor de una celda en un número flotante"""
    # valor = 0
    try:
        valor = float(celda.value)
    except ValueError:
        valor = 0
    return valor

In [40]:
def get_data_excel(file):
    try: 
        wb = load_workbook(filename = file, data_only=True)
    except Exception: 
        raise BadRequest(e)
    ws = wb.active
    remesa = ws['J7'].value[8:15]
    observaciones = ws['A41'].value
    macs = {}
    
    for row in range(9, 22):
        mac = ws[row]
        try:
            _mac = str(mac[0].value)
            if _mac[:3] == '290':
                macs[_mac] = {
                    'distrito': _mac[2:4],
                    'type': mac[1].value,
                    'days_worked': mac[2].value,
                    'hours_worked': mac[3].value,
                    'setup': mac[4].value,
                    'tramites_aplicados': mac[5].value,
                    'cards_by_update': mac[6].value,
                    'total_atenciones': mac[7].value,
                    'numbers_by_day': mac[8].value,
                    'numbers_by_machine_by_day': mac[9].value,
                    'credenciales_recibidas': mac[11].value
                }
        except ValueError:
            pass
    
    return remesa, observaciones, macs

r, o, macs = get_data_excel(r07b)

In [41]:
print(r)

2021_07


In [39]:
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
INICIO = '2021-01-18'
FINAL = '2021-12-31'

from apps.productividad.models import Reporte
reportes = Reporte.objects.filter(fecha_corte__gte=INICIO, fecha_corte__lte=FINAL).order_by('fecha_corte')
reportes.values()

<QuerySet [{'id': 133, 'created': datetime.datetime(2021, 7, 24, 1, 47, 15, 283292, tzinfo=<UTC>), 'updated': datetime.datetime(2021, 7, 24, 1, 48, 27, 576021, tzinfo=<UTC>), 'fecha_corte': datetime.date(2021, 2, 7), 'remesa': '2021-06', 'notas': 'En está remesa no se presento ninguna incidencia que suspendiera el servicio en los Módulos de Atención Ciudadana.', 'archivo': 'D:/Users/javier/Projects/cerebro/src/apps/media/productividad/remesa-20210207.xlsx', 'usuario_id': 8}]>